In [1]:
import pandas as pd
import mlflow
import mlflow.xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv('data/data-cleaned.csv')

# Define X and y if not already defined
X = df.drop("output", axis=1)
y = df["output"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# # Set up MLflow tracking URI
# mlflow.set_tracking_uri("http://localhost:5000")
# mlflow.set_experiment("heart_attack_training")

# with mlflow.start_run():
#     # Model training with XGBoost
#     model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
#     model.fit(X_train, y_train)

#     # Model evaluation
#     predictions = model.predict(X_test)
#     accuracy = accuracy_score(y_test, predictions)

#     # Log model parameters, metrics, and model itself
#     mlflow.log_param("model_type", "XGBoost")
#     mlflow.log_metric("accuracy", accuracy)
#     mlflow.xgboost.log_model(model, "model")


In [3]:
# Set up MLflow tracking URI
mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("heart_attack_training")

with mlflow.start_run():
    # Model training with XGBoost
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    # Model evaluation
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    # Log model parameters, metrics, and model itself
    mlflow.log_param("model_type", "XGBoost")
    mlflow.log_metric("accuracy", accuracy)
    
    # Specify the path for saving the model
    mlflow.xgboost.log_model(model.get_booster(), artifact_path="models")


2024/11/10 13:06:55 INFO mlflow.tracking.fluent: Experiment with name 'heart_attack_training' does not exist. Creating a new experiment.
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
